<a href="https://colab.research.google.com/github/LunaticRuri/cpng-reviews-nlp/blob/master/%EA%B0%9C%EC%84%A0%EB%90%9C_%EB%A6%AC%EB%B7%B0_%EA%B8%B0%EB%B0%98_%EC%83%81%ED%92%88_%EC%B6%94%EC%B2%9C_%EC%8B%9C%EC%8A%A4%ED%85%9C_%EC%A0%9C%EC%95%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 간단하지만 더 나아진 상품 추천 시스템 

## 들어가며
지금까지 우리는 Doc2Vec 모델을 통해 문서 자체를 벡터 공간에 임베딩함으로써 리뷰 데이터를 기반으로 한 추천 시스템을 구현할 수 있었다.

그러나 이 리뷰 기반 시스템은 리뷰에서 드러난 상품의 특성을 종합적으로 반영하지만, 그것들을 개별적으로 반영하지는 못한다는 한계가 있다.

이런 이유로, 최근 가장 주목 받고 있는 BERT를 활용하여 상품 추천 시스템을 구현하고자 한다.

ETRI KoBERT는 승인 대기가 오래 걸려서 일단 https://github.com/snunlp/KR-SBERT 로 진행한다.

문장 임베딩에 특화된 SBERT로 몇 문장의 코사인 유사도를 측정해보자...

In [2]:
!pip install -U transformers sentence-transformers kss
!pip install -U torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 79 kB 7.9 MB/s 
     |████████████████████████████████| 42.4 MB 1.2 MB/s 
     |████████████████████████████████| 6.6 MB 44.2 MB/s 
     |████████████████████████████████| 596 kB 56.4 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 57.8 MB/s 
     |████████████████████████████████| 175 kB 66.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=27b158f5020f69df13478d70617d8af7b78a84336364d1db6db0fcc551930ed4
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
  Created wheel for kss: filename=kss-3.4.2-py3-none-any.whl size=42448069 sha256=d4fa3234a2478621d418805ede431a192ba4a46436ddff7743c85c7b569da6e7
  Store

In [3]:
from sentence_transformers import SentenceTransformer, util
import torch
# https://github.com/snunlp/KR-SBERT의 pretrained model 사용
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS') 

Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/707 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/967k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/394 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336k [00:00<?, ?B/s]

In [4]:
def print_similarities(sentences):
    vectors = model.encode(sentences) # encode sentences into vectors
    similarities = util.cos_sim(vectors, vectors) # compute similarity between sentence vectors
    print(similarities)

In [ ]:
sentences = ['가격이 저렴하다', '값이 싸다', '가격이 비싸다', '음질이 깨끗하다']
print_similarities(sentences)

tensor([[1.0000, 0.9201, 0.7356, 0.5097],
        [0.9201, 1.0000, 0.7373, 0.5134],
        [0.7356, 0.7373, 1.0000, 0.3178],
        [0.5097, 0.5134, 0.3178, 1.0000]])


In [ ]:
sentences = ['디자인이 아름답다', '모양이 예쁘다', '디자인이 촌스럽다', '국물이 짜다']
print_similarities(sentences)

tensor([[1.0000, 0.7730, 0.6020, 0.3241],
        [0.7730, 1.0000, 0.4116, 0.3483],
        [0.6020, 0.4116, 1.0000, 0.1833],
        [0.3241, 0.3483, 0.1833, 1.0000]])


In [ ]:
sentences = ['면도날이 날카롭다', '면도날이 잘 든다', '면도날이 무디다', '음질이 깨끗하다']
print_similarities(sentences)

tensor([[1.0000, 0.7909, 0.8123, 0.4160],
        [0.7909, 1.0000, 0.8082, 0.5184],
        [0.8123, 0.8082, 1.0000, 0.3827],
        [0.4160, 0.5184, 0.3827, 1.0000]])


같은 측면에서 비슷한 의미이면 유사도가 높고 반대 의미라도 다른 측면의 경우보다는 대체로 유사도가 높다

In [ ]:
sentences = ['저렴한 라면', '가성비 라면','비싼 라면', '싼 라면']
print_similarities(sentences)

tensor([[1.0000, 0.8220, 0.9138, 0.9255],
        [0.8220, 1.0000, 0.7889, 0.8420],
        [0.9138, 0.7889, 1.0000, 0.9181],
        [0.9255, 0.8420, 0.9181, 1.0000]])


**우리의 직관과 많이 다르다.**

이유를 추정해보면 실제로 '싸다'와와 '비싸다'는 사람에게는 정반대의의 단어이지만, 기계에게는 비슷한 맥락에서 사용되는 경우가 많기 때문이지 않을까 한다.

실제 리뷰 데이터를 넣어보자.

In [ ]:
R1 = "쿠팡에서 가장 저렴한 라면 늘 먹던 라면 말고.. 오늘은 뭔가 새로운 라면이 먹고싶더라구요 쿠팡에서 라면을 검색해서 결과에서 낮은 가격순으로 정렬했더니 이제품이 가장 저렴하더라구요 물론 사리면이 가장 저렴하다고도 할수있겠지만.. 그건 스프가 안들은거고 스프가 있는 라면중.. 가장 저렴한 라면"
R2 = "가격이 너무 비싸요 진심 실망입니다 맛이 없어요 다시 구매안함 광고보고 혹시나 하는 맘으로 비싸기도 하고 배송비까지해서 구매했는데 기대보다는 가격에 비해서 실망했네요"    

sentences = ['가격이 싸다', R1, R2]
print_similarities(sentences)

tensor([[1.0000, 0.1433, 0.2194],
        [0.1433, 1.0000, 0.4484],
        [0.2194, 0.4484, 1.0000]])


r1은 가격이 싸다는 리뷰, r2는 가격이 비싸다는 리뷰이다.

우리의 직관은 '가격이 싸다'라는 문장이 r2보다는 r1와 가깝다는 것인데,


*   "가격이 싸다" <-> r1 0.1433
*   "가격이 싸다" <-> r2 0.2194

유사도가 매우 낮을 뿐만이 아니라, 그 수치도 반대로 나왔다.

그렇다면 아래의 단순한 phrase의 나열로 이루어진 문장을 보자.



In [ ]:
r1 = "가격이 싸다 가격이 저렴하다 가격이 싸다 가격이 저렴하다 가격이 싸다 가격이 저렴하다 가격이 싸다 가격이 저렴하다"
r2 = "가격이 싸다 가격이 비싸다"

sentences = ['가격이 싸다', r1, r2]
print_similarities(sentences)

tensor([[1.0000, 0.5485, 0.8138],
        [0.5485, 1.0000, 0.6751],
        [0.8138, 0.6751, 1.0000]])


r1보다 r2의 "가격이 싸다" 쿼리 문장과의 유사도가 높게 나온다.

지금까지의 예시를 통해 알 수 있는 것은



1.   문장의 길이에 따라 유사도가 영향을 많이 받는다. (길이가 길어질수록, 즉, 쿼리 문장과 관련이 없는 다른 단어들이 많이 포함될수록) 유사도는 낮아진다.
2.   SBERT에서는 어떠하다는 성질의 방향성을 잘 알지 못한다. (싼 라면 <-> 비싼 라면 0.9181)


그런데 리뷰 데이터 기반 추천 시스템 구현을 위해서는 r1, r2와 "가격이 싸다"는 문장과의 유사도가 r1 > r2 이어야 한다...



## 선행연구와 본 프로젝트의 의미

상업적 활용 가능성 때문인지, 리뷰를 기반으로한 추천 시스템에 대해 많은 연구들이 이루어져왔다.

그렇지만, 꽤나 많은 연구가 Aspect 추출에서 수작업이나 그에 준하는 방법을 사용한다는 한계를 보였다.

또한 Word2vec과 같은 워드 임베딩, 감성 사전 등을 이용한 감성 분석 연구가 많았다.

부족한 시간과 겸손한 지식으로 인해 관련 필드의 주요 논문을 읽고 제대로 이해하지는 못했지만, 문장 단위 임베딩을 이용하여 한국어 리뷰 데이터를 이용해서 연관 상품을 추출하는 것은 예상 외로 많지는 않았다.

앞으로 보이고자 하는 탐구는 문장 임베딩 수준에서 문서(본 탐구에서는 상품)의 특성을 나타낸다는 점에서 SBERT를 이용한 토픽 모델링 BERTopic과 비슷하다. 그러나 BERTopic과는 도출하고자 하는 결과가 다르다.

솔직히 관련 연구가 너무 많이 이루어져서 앞으로 진행하고자 하는 방법도 누군가가 이미 비슷하게 한 것이 있지 않을까 자신이 없다. 

그렇지만 제시하고자 하는 메소드는 최신의 연구와는 달리 상당히 단순하여 학부생 수준에서 나름의 결과를 낼 수 있는 방법일 것이라 예상된다.



## 아이디어

우리의 아이디어는 의존 관계 분석에서 시작한다.

리뷰 하나에는 하나의 target에 대한 평가만이 존재한다고 가정했을 때,

리뷰에서 어떤 상품의 특성을 드러내는 구(phrase)에는 

**측면과 기술, 다시 말해 Aspect와 Description(이는 Sentiment로 해석될 수 있다)라는 두 가지 요소가 있다.**


한국어에서 이를 phrase로 표현하는 방식은 크게 두 가지로 나눌 수 있다.

1. *무엇이 어떠하다, 어찌하다*

    * 가격이 싸다 -> 측면: 가격, 기술: 싸다

    * 디자인이 아름답다 -> 측면: 디자인, 기술: 아름답다

    * 조미료가 덜 들어가다 -> 측면: 조미료, 기술: 덜 들어가다

2. *어떠한, 어찌한 무엇이다*
    
    * 상한 두부 -> 측면: 두부, 기술: 상한

    * 날아가는 밥알 -> 측면: 밥알, 기술: 날아가는

    * 맛있는 라면 -> 측면: 라면, 기술: 맛있는

*   '맛있다'라는 phrase는 측면: "", 기술: 맛있다  

*   마찬가지 방법으로 '역시 삼성'이라는 phrase는 측면: 삼성, 기술: ""


이를 바탕으로 만약 우리가

1.  SBERT를 이용해 (“가격이 저렴하다”,“값이 싸다”), (“디자인이 아름답다”, “모양이 예쁘다”)와 같은 문장 쌍의 유사도를 잘 측정할 수 있고,

2. “싸다”와 “비싸다”의 의미와 같이 기술 부분의 의미를 긍부정에 따라 잘 분별할 수 있고,

3. 여러 리뷰 데이터의 합으로 구성되는 상품 특성을 “가격이 싸다”, “색깔이 예쁘다”와 같은 쿼리 문장을 통해, 찾고자 하는 특성이 잘 포함되어있는 상품을 찾을 수 있다면

감성 분류의 가장 큰 난관중의 하나인 Aspect Extraction을 굳이 수행할 필요가 없다고 생각했다.

왜냐하면, 상품의 특성을 리뷰 phrase들의 집합으로 보고 어떠한 쿼리 phrase가 주어졌을 때, 모든 구성 phrase들에 대해서 각각 쿼리 phrase와의 유사도를 계산해, 그 가중치로 필터링한 평균을 상품의 그 쿼리 phrase 측면의 특성이라고 간주할 수 있으리라 여겨지기 때문이다.

In [ ]:
r1 = "가격이 싸다 가격이 저렴하다 가격이 싸다 가격이 저렴하다 가격이 싸다 가격이 저렴하다 가격이 싸다 가격이 저렴하다"
r2 = "가격이 싸다 가격이 비싸다"
a = "가격이 싸다"
b = "가격이 저렴하다"
c = "가격이 비싸다"
sentences = [a, b, c]
print_similarities(sentences)

tensor([[1.0000, 0.9439, 0.7809],
        [0.9439, 1.0000, 0.7356],
        [0.7809, 0.7356, 1.0000]])


구체적인 수치로 예를 들면, 여기서 (a, a) = 1, (a,b) = 0.9439, (a, c) = 0.7809일 때 산술평균으로 구한 쿼리 phrase a에 대한 유사도는 아래와 같다.

*   r1 = (1 + 0.9439 + 1 + 0.9439 + 1 + 0.9439 + 1 + 0.9439)*(1/8) = 0.97195

*   r2 = (1 + 0.7809)*(1/2) = 0.89045

r1이 쿼리 phrase a에에 대해 더 유사하다는 결론을 낼 수 있다.



리뷰 데이터에는'국물이 짜다'와 같은 집중하고자 하는 측면과 상관이 없는 다른 phrase가 많을 것이다...

In [ ]:
sentences = ['가격이 싸다', '값이 비싸다', '국물이 짜다']
print_similarities(sentences)

tensor([[1.0000, 0.7629, 0.3045],
        [0.7629, 1.0000, 0.2524],
        [0.3045, 0.2524, 1.0000]])


In [ ]:
sentences = ['색깔이 예쁘다', '디자인이 별로다', '사이즈가 넉넉하다']
print_similarities(sentences)

tensor([[1.0000, 0.3939, 0.4548],
        [0.3939, 1.0000, 0.2593],
        [0.4548, 0.2593, 1.0000]])


디자인과 색깔 측면이 별개라는 위의 결과를 수용?하면

In [ ]:
sentences = ['색깔이 예쁘다', '면발이 쫄깃하다', '사이즈가 넉넉하다', '음질이 깨끗하다', '값이 싸다']
print_similarities(sentences)

tensor([[1.0000, 0.4699, 0.4548, 0.5070, 0.5681],
        [0.4699, 1.0000, 0.3249, 0.4675, 0.3888],
        [0.4548, 0.3249, 1.0000, 0.3915, 0.5915],
        [0.5070, 0.4675, 0.3915, 1.0000, 0.5134],
        [0.5681, 0.3888, 0.5915, 0.5134, 1.0000]])


서로 다른 측면의 phrase는 서로 낮은 유사도를 가지지만

In [5]:
sentences = ['값이 싸다', '가격이 비싸다', '가격이 저렴하다', '음질이 깨끗하다']
print_similarities(sentences)
sentences = ['값 싸', '가격 비싸', '가격 저렴', '음질 깨끗']
print_similarities(sentences)

tensor([[1.0000, 0.7373, 0.9201, 0.5134],
        [0.7373, 1.0000, 0.7356, 0.3178],
        [0.9201, 0.7356, 1.0000, 0.5097],
        [0.5134, 0.3178, 0.5097, 1.0000]])
tensor([[1.0000, 0.7765, 0.8542, 0.4155],
        [0.7765, 1.0000, 0.7363, 0.3189],
        [0.8542, 0.7363, 1.0000, 0.4160],
        [0.4155, 0.3189, 0.4160, 1.0000]])


서로 비슷한 측면의 phrase들끼리는 상대적으로 높은 유사도를 가질 수 있을 것이라 기대할 수 있을 것 같다.

게다가, 적당히 어근?만 남기면 근소하게 더 잘 분별하는 듯하다.

In [6]:
sentences = ['상태가 좋다', '상태가 나쁘다', '상태가 별로다', '사이즈가 넉넉하다', '가격이 싸다']
print_similarities(sentences)
sentences = ['상태 좋', '상태 나쁘', '상태 별로', '사이즈 넉넉', '가격 싸']
print_similarities(sentences)

tensor([[1.0000, 0.5779, 0.4370, 0.4858, 0.4608],
        [0.5779, 1.0000, 0.6713, 0.2038, 0.1388],
        [0.4370, 0.6713, 1.0000, 0.2255, 0.1488],
        [0.4858, 0.2038, 0.2255, 1.0000, 0.6187],
        [0.4608, 0.1388, 0.1488, 0.6187, 1.0000]])
tensor([[1.0000, 0.6310, 0.4865, 0.3710, 0.4552],
        [0.6310, 1.0000, 0.6944, 0.1748, 0.2589],
        [0.4865, 0.6944, 1.0000, 0.1491, 0.1745],
        [0.3710, 0.1748, 0.1491, 1.0000, 0.5255],
        [0.4552, 0.2589, 0.1745, 0.5255, 1.0000]])


우리의 예상보다 현실은 조금 더 잔혹한 것 같지만 어쩔 수 없다... 이게 좋다, 나쁘다, 별로다 같은 직접적인 표현은 임베딩에 의미를 더 포함시키는 것 같기도 하고... 일단 가정이 대체로 맞다는 믿음 하에 그대로 진행해보자.

그렇다면 이런 phrase에 대해 긍부정으로 분류 작업을 수행하고, 서로 유사도가 높으면서 감정이 정반대로 파악되는 것들을 반대의 의미로 생각할 수도 있다.

또한 감정 분류 시 긍정, 부정에서 골고루 나온 phrase는 상품의 특성과는 관계가 없다고 해석한다면, 조금 더 명확한 결과를 도출할 수 있을 것이라고 예상한다.

우리가 측면과 기술로 이루어진 phrase나 기술로만 이루어진 phrase에만 집중하고,

이것과 함께, 리뷰를 phrase로 잘게 쪼개고, 구조화시키면 더 좋은 결과가 나온다는 예측을 이전에 살펴본 실제 리뷰데이터에 적용하면,

In [ ]:
print("감성 수치 적용 x")

r1 = ["라면 저렴", "라면 먹", "라면 새로운", "라면 먹", "라면 검색", "가격 낮은"]
r2 = ["가격 비싸", "진심 실망", "맛 없", "구매 안함", "광고 보", "비싸"]

def dividing_similarities(query_phrase, sentences):
    sentences.insert(0, query_phrase)
    vectors = model.encode(sentences) # encode sentences into vectors
    similarities = util.cos_sim(vectors, vectors) # compute similarity between sentence vectors
    
    value_sum = 0
    print(similarities[1:,:1])

    return torch.sum(similarities[1:,:1]).item()/(len(sentences)-1)

query = '가격 싸' # + 감정 

print("r1: ", dividing_similarities(query, r1))
print("r2: ", dividing_similarities(query, r2))

print("\n")
print("감성 수치 적용 O 단순화를 위해서 query와 같은 감정이면 +1, 중립이면 0, 반대면 -1; 실제로는 분류 확률? 적용")

r1 = [("라면 저렴", 1), ("라면 먹", 0), ("라면 새로운", 0), ("라면 먹", 0), ("라면 검색", 0), ("가격 낮은",1)]
r2 = [("가격 비싸", -1), ("진심 실망", -1), ("맛 없", -1), ("구매 안함", -1), ("광고 보", 0), ("비싸", -1)]

def dividing_similarities_with_sentiment(query_phrase, sentences_sent):
    
    sentences = [elem[0] for elem in sentences_sent]
    sent_coef = torch.tensor([elem[1] for elem in sentences_sent]).to(torch.float32)

    sentences.insert(0, query_phrase)

    vectors = model.encode(sentences)
    similarities = util.cos_sim(vectors, vectors).to(torch.float32)
    
    # 유사도 벡터에 감성 계수 벡터를 곱해준다
    value_sum = 0
    simil_sent = torch.flatten(similarities[1:,:1]).T @ sent_coef
    
    return torch.sum(simil_sent).item()/(len(sentences)-1)

print("r1: ", dividing_similarities_with_sentiment(query, r1))
print("r2: ", dividing_similarities_with_sentiment(query, r2))

감성 수치 적용 x
tensor([[0.4769],
        [0.0962],
        [0.1661],
        [0.0962],
        [0.0795],
        [0.7854]])
r1:  0.28339622418085736
tensor([[0.8301],
        [0.0562],
        [0.4506],
        [0.2618],
        [0.2090],
        [0.7724]])
r2:  0.4300195376078288


감성 수치 적용 O 단순화를 위해서 query와 같은 감정이면 +1, 중립이면 0, 반대면 -1; 실제로는 분류 확률? 적용
r1:  0.21038788557052612
r2:  -0.39518574873606366


r1과 r2의 명확한 차이를 얻을 수 있다.



### 아이디어의 한계점

방금 전 긍부정이 골고루 나온 phrase는 상품의 특성과 관계없다고 했는데, 이는 잘못된 가정일 가능성이 매우 높다.

예를 들어 다음을 보면,

In [ ]:
sentences = ['국물이 얼큰하다', '라면이 맵다', '국물이 순하다']
print_similarities(sentences)

tensor([[1.0000, 0.6643, 0.8602],
        [0.6643, 1.0000, 0.7557],
        [0.8602, 0.7557, 1.0000]])


문장의 유사도가 각각

국물이 얼큰하다 - 국물이 맵다 : 0.8574

국물이 순하다 - 국물이 맵다 : 0.9133

인데, '순하다'는 몰라도, 맵다에 감정적인 편향이 있을까?

설사 있다고 해도, 그것이 '순하다'와 반대 감정이라는 보장이 있을까?

제시한 방법은 속성 부분이 긍부정에서 명확히 갈리는 속성(Ex: 배송, 신선도, 내구도, 가격 등)이거나, 기술 부분이 명확하게 감정을 드러내는 경우(좋다, 나쁘다 등)가 아니라면, 좋은 성능을 기대하기 어려울 수 있다.

이러한 점에 대해서는 사실 감정 분석의 범위를 벗어난 것일 수도 있다.

예를 들어, 매운 정도를 별점으로 나타내는 것은 일단 소비자의 만족도와는 별개이기 때문이다.

이를 Word2Vec으로 단어 임베딩하여 해결해보려고도 하였으나, 마찬가지로 



### 다른 아이디어 방향

굳이 SBERT로 매달리지 말고 word2vec 이용하는 것도 방법이다

##유용성

이전에 구현한 Doc2Vec 모델에서 **(2437845, '농심 사리곰탕면 봉지 110g')**의 상위 10개 유사 상품은 다음과 같다. 
    
    (2266323, '삼양라면 120g') 0.6619757413864136

    ('7958974', '농심 신라면') 0.6579278707504272

    ('2438000', '팔도 꼬꼬면') 0.6411586403846741

    (14341, '삼양 담백한 된장라면 120g') 0.6134355068206787

    ('6247752662', '삼양 손칼국수') 0.5990796089172363

    ('2451953', '생생우동 봉지') 0.5971024036407471

    (1067207282, '오뚜기 김치라면 120g') 0.5970842242240906

    ('86478', '농심 감자면 117g') 0.5866415500640869

    ('14111627', '오뚜기 진라면 매운맛 멀티팩 120g') 0.5843058824539185

    (14335, '삼양 수타면 120g') 0.5724368095397949

물론 여기서 최소한 '인스턴트 봉지 라면'이 추천 상품으로 제시되었다는 점,

사리곰탕면이 '하얀 국물 라면'이라는 걸 고려했을 때, 꼬꼬면, 된장라면, 손칼국수, 생생우동이 상위권에 위치한다는 점은 주목할만하다.

하지만, 이는 리뷰의 총체적인 특성만을 고려한 결과이다.

그렇다면, 여기서 '가격이 싸다', '면발이 쫄깃하다', '국물이 시원하다' 등과 같은 쿼리 phrase로 이 상품들을 한번 더 필터링하여 추천 상품을 제시할 수 있다면 고객들이 조금 더 만족할 수 있는 시스템이 될 수 있지 않을까 한다.

한편, 일반 상품 검색과 제시된 연관 상품 추천 시스템의 구별이 문제된다. 

일단, 판매자가 제시한 특성과 소비자가 생산한 리뷰 데이터에서 추출해낸 특성은 본질적으로 다르다. (쿠팡의 검색 시스템에 대해서는 자세히 알지 못하기에 확실한 것은 아니다.)

또한, 이러한 쿼리 phrase를 사용자의 특성으로 귀속시켜 사용자 맞춤형 추천 시스템의 구현에도 도움을 줄 수 있을 것 같다는 예상을 조심스럽게 해본다.


In [10]:
sentences = ['국물', '국물이 짜다', '가격이 싸다']
print_similarities(sentences)

tensor([[1.0000, 0.7837, 0.1833],
        [0.7837, 1.0000, 0.3045],
        [0.1833, 0.3045, 1.0000]])
